Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [30]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Hidden layer.
    hidden_layer_size = 1024
    weights_hidden = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))

    # Output layer.
    weights_output = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    biases_output = tf.Variable(tf.zeros([num_labels]))
    
    # Train.
    layer_hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)  
    logits = tf.matmul(layer_hidden, weights_output) + biases_output
    
    beta = 0.001
    regularization = tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(biases_hidden) + tf.nn.l2_loss(
        weights_output) + tf.nn.l2_loss(biases_output)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
                         ) + beta * regularization

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden)
    valid_logits = tf.matmul(valid_hidden, weights_output) + biases_output
    valid_prediction = tf.nn.softmax(valid_logits)

    test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden)
    test_logits = tf.matmul(test_hidden, weights_output) + biases_output
    test_prediction = tf.nn.softmax(test_logits)


In [31]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

# Improved test accuracy: from 90.7% to 93.7%

Initialized
Minibatch loss at step 0: 684.733826
Minibatch accuracy: 14.8%
Validation accuracy: 32.8%
Minibatch loss at step 500: 206.395905
Minibatch accuracy: 78.9%
Validation accuracy: 78.0%
Minibatch loss at step 1000: 118.537651
Minibatch accuracy: 74.2%
Validation accuracy: 81.7%
Minibatch loss at step 1500: 69.753937
Minibatch accuracy: 87.5%
Validation accuracy: 83.2%
Minibatch loss at step 2000: 41.544003
Minibatch accuracy: 90.6%
Validation accuracy: 84.5%
Minibatch loss at step 2500: 25.517529
Minibatch accuracy: 84.4%
Validation accuracy: 84.5%
Minibatch loss at step 3000: 15.595974
Minibatch accuracy: 85.9%
Validation accuracy: 86.2%
Minibatch loss at step 3500: 9.688896
Minibatch accuracy: 85.9%
Validation accuracy: 86.6%
Minibatch loss at step 4000: 5.953754
Minibatch accuracy: 93.0%
Validation accuracy: 87.4%
Minibatch loss at step 4500: 3.856411
Minibatch accuracy: 86.7%
Validation accuracy: 87.0%
Minibatch loss at step 5000: 2.507466
Minibatch accuracy: 89.8%
Validati

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [32]:
batch_size = 32

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Hidden layer.
    hidden_layer_size = 1024
    weights_hidden = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))

    # Output layer.
    weights_output = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    biases_output = tf.Variable(tf.zeros([num_labels]))
    
    # Train.
    layer_hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)  
    logits = tf.matmul(layer_hidden, weights_output) + biases_output
    
    beta = 0.001
    regularization = tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(biases_hidden) + tf.nn.l2_loss(
        weights_output) + tf.nn.l2_loss(biases_output)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
                         ) + beta * regularization

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden)
    valid_logits = tf.matmul(valid_hidden, weights_output) + biases_output
    valid_prediction = tf.nn.softmax(valid_logits)

    test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden)
    test_logits = tf.matmul(test_hidden, weights_output) + biases_output
    test_prediction = tf.nn.softmax(test_logits)


In [33]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

# Overfitting
# Decreased batch size: from 128 to 32
# Decreased test accuracy: from 93.7% to 88%

Initialized
Minibatch loss at step 0: 686.687561
Minibatch accuracy: 12.5%
Validation accuracy: 21.1%
Minibatch loss at step 500: 207.295547
Minibatch accuracy: 78.1%
Validation accuracy: 75.6%
Minibatch loss at step 1000: 121.615547
Minibatch accuracy: 84.4%
Validation accuracy: 75.5%
Minibatch loss at step 1500: 73.309608
Minibatch accuracy: 81.2%
Validation accuracy: 77.2%
Minibatch loss at step 2000: 44.222198
Minibatch accuracy: 81.2%
Validation accuracy: 78.5%
Minibatch loss at step 2500: 28.033468
Minibatch accuracy: 65.6%
Validation accuracy: 81.0%
Minibatch loss at step 3000: 16.800970
Minibatch accuracy: 93.8%
Validation accuracy: 82.5%
Minibatch loss at step 3500: 11.039650
Minibatch accuracy: 75.0%
Validation accuracy: 83.1%
Minibatch loss at step 4000: 6.369487
Minibatch accuracy: 87.5%
Validation accuracy: 84.0%
Minibatch loss at step 4500: 4.005630
Minibatch accuracy: 90.6%
Validation accuracy: 82.3%
Minibatch loss at step 5000: 2.569186
Minibatch accuracy: 96.9%
Validat

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [128]:
# Decreased accuracy.
# Only efficient in overfitting cases.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [27]:
batch_size = 128
hidden_layer1_size = 1024
hidden_layer2_size = 1024
hidden_layer3_size = 512
hidden_layer4_size = 256
beta = 0.08

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    keep_rate = tf.placeholder(tf.float32)
    
    # 3 Hidden layers and 1 output layer.
    weights_hidden1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer1_size], stddev = 0.05))
    weights_hidden2 = tf.Variable(tf.truncated_normal([hidden_layer1_size, hidden_layer2_size], stddev = 0.05))
    weights_hidden3 = tf.Variable(tf.truncated_normal([hidden_layer2_size, hidden_layer3_size], stddev = 0.05))
    weights_hidden4 = tf.Variable(tf.truncated_normal([hidden_layer3_size, hidden_layer4_size], stddev = 0.05))
    weights_output = tf.Variable(tf.truncated_normal([hidden_layer4_size, num_labels], stddev = 0.08))
    biases_hidden1 = tf.Variable(tf.zeros([hidden_layer1_size])) 
    biases_hidden2 = tf.Variable(tf.zeros([hidden_layer2_size]))
    biases_hidden3 = tf.Variable(tf.zeros([hidden_layer3_size]))
    biases_hidden4 = tf.Variable(tf.zeros([hidden_layer4_size]))
    biases_output = tf.Variable(tf.zeros([num_labels]))

    def fourLayerNetwork(dataset):
        hidden_layer1 = tf.nn.relu(tf.matmul(dataset, weights_hidden1) + biases_hidden1)   
        hidden_layer1 = tf.nn.dropout(hidden_layer1, keep_rate)
        hidden_layer2 = tf.nn.relu(tf.matmul(hidden_layer1, weights_hidden2) + biases_hidden2)
        hidden_layer2 = tf.nn.dropout(hidden_layer2, keep_rate)
        hidden_layer3 = tf.nn.relu(tf.matmul(hidden_layer2, weights_hidden3) + biases_hidden3)       
        hidden_layer3 = tf.nn.dropout(hidden_layer3, keep_rate)
        hidden_layer4 = tf.nn.relu(tf.matmul(hidden_layer3, weights_hidden4) + biases_hidden4)       
        hidden_layer4 = tf.nn.dropout(hidden_layer4, keep_rate)
        logits = tf.matmul(hidden_layer4, weights_output) + biases_output
        return logits

    # Train, Valid, and Test.
    logits_train = fourLayerNetwork(tf_train_dataset)
    logits_valid = fourLayerNetwork(tf_valid_dataset)
    logits_test = fourLayerNetwork(tf_test_dataset)

    # Loss function.
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits_train, tf_train_labels)) + (
        beta * tf.nn.l2_loss(weights_output))

    # Optimizer.
    global_step = tf.Variable(0)
    # tf.train.exponential_decay(learning_rate, global_step, decay_steps, decay_rate, staircase=False, name=None)
    learning_rate = tf.train.exponential_decay(0.3, global_step, 3500, 0.86, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits_train)
    valid_prediction = tf.nn.softmax(logits_valid)
    test_prediction = tf.nn.softmax(logits_test)    
    

In [28]:
num_steps = 100000

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_rate : 0.75}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(train_prediction.eval(
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_rate : 1.0}), 
            batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(feed_dict = {keep_rate : 1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(
        test_prediction.eval(feed_dict = {keep_rate : 1.0}), test_labels))


Initialized
Minibatch loss at step 0: 2.947919
Minibatch accuracy: 35.9%
Validation accuracy: 24.1%
Minibatch loss at step 1000: 0.585540
Minibatch accuracy: 86.7%
Validation accuracy: 85.0%
Minibatch loss at step 2000: 0.518836
Minibatch accuracy: 93.0%
Validation accuracy: 84.4%
Minibatch loss at step 3000: 0.443016
Minibatch accuracy: 91.4%
Validation accuracy: 87.6%
Minibatch loss at step 4000: 0.407132
Minibatch accuracy: 93.8%
Validation accuracy: 87.9%
Minibatch loss at step 5000: 0.471312
Minibatch accuracy: 90.6%
Validation accuracy: 88.6%
Minibatch loss at step 6000: 0.493784
Minibatch accuracy: 87.5%
Validation accuracy: 88.9%
Minibatch loss at step 7000: 0.799120
Minibatch accuracy: 87.5%
Validation accuracy: 88.6%
Minibatch loss at step 8000: 0.597585
Minibatch accuracy: 89.1%
Validation accuracy: 89.0%
Minibatch loss at step 9000: 0.525339
Minibatch accuracy: 91.4%
Validation accuracy: 89.1%
Minibatch loss at step 10000: 0.389182
Minibatch accuracy: 93.8%
Validation accur